### Import libraries

In [1]:
#Use with python/3.6.0 and tensorflow/1.10

import math
import sys
from sklearn.linear_model import LogisticRegression
import random
import os
import csv
import json
from collections import defaultdict
import gensim
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn import metrics
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from glove import Corpus, Glove
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score
from tensorflow.python.keras.constraints import maxnorm
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error
import itertools

from bert_serving.client import BertClient
bc = BertClient()

### Load data, define target diagnosis

In [2]:
## change data path to where the mimic sequences are located
data_path = "/users/emily/Documents/mimic_seq/"
random.seed(1)

def get_file_list(data_path):
    # list for files
    train_files = []
    valid_files = []
    full_data_files = []

    for i in range(7):
        train_files.append(data_path + 'test_'+str(i))
    for i in range(7,10):
        valid_files.append(data_path + 'test_'+str(i))
    for i in range(10):
        full_data_files.append(data_path + 'test_'+str(i))
    
    return train_files, valid_files, full_data_files

## define target diagnosis

#test = ['d_041','d_250','d_401','d_414','d_424','d_427','d_530','d_585']
test = ['d_585']

In [4]:
train_files, valid_files, full_data_files = get_file_list(data_path)

### Generate embeddings dataframe

In [7]:
for q in range(len(test)):
    
    ### Take raw mimic sequence data and generate a list of medical concepts for each patient
    
    def create_disease_sentences(files):
        count = -1
        sentences = []
        demographic = []
        final_dx = []
        disease_prev = []

        for i in files:
            with open(i) as f:
                for s in f:
                    count = count + 1
                    demographic.append(s.split("|")[1].split(" "))
                    sentences.append(s.split("|")[2].split(" ") +
                                     s.split("|")[3].replace("\n", "").split(" "))
                    next_diags = s.split("|")[0].split(",")
                    final_dx.append(next_diags)
                    prev_diags = [e for e in s.split("|")[2].split(" ") if e.startswith("d_")]

                    if test[q] in prev_diags: # if disease exit in the previous diagnosis
     
                            disease_prev.append(1)
                    
                    else:
                        disease_prev.append(0)

        ## create a vector indicating which patients have disease as a final diagnosis
        disease_final_ori = []
        for sublist in final_dx:
            if test[q] in sublist:
                disease_final_ori.append(1)
            else:
                disease_final_ori.append(0)


        return sentences, demographic, final_dx, disease_prev, disease_final_ori
    sentences, demographic, final_dx_train, disease_prev, disease_train_ori = create_disease_sentences(train_files)
    
    ### Identify unique medical concepts in the data
    def generate_bert_dict(sentences):

        flat_sentences = list(itertools.chain(*sentences))

        events = np.unique(flat_sentences)
        events = np.ndarray.tolist(events)

        bert_dict = []
        bertdict = []

        return events, bert_dict, bertdict
    events, bert_dict, bertdict = generate_bert_dict(sentences)

    ## Get the vectors corresponding to the entire patient history
    
    def generate_patient_disease_vectors(files, events, bert_dict):
        count = -1
        count2 = 0

        patient_seq_all_disease = []
        patient_seq_all_disease_1 = []
        patient_sen_disease = []
        exclude = []

        _, _, _, disease_prev, disease_final_ori = create_disease_sentences(files)

        for i in files: 
            with open(i) as f:
                for line in f:
                    count = count + 1
                    patient_seq = []
                    feed_events = line.split("|")[2].split(" ")

                    if disease_prev[count] == 1: # Only include events before the target diagnosis
                        target = feed_events.index(test[q])
                        feed_events = feed_events[:target]
                        if feed_events == []:
                            count2 = count2+1
                            exclude.append(1) #exclude patients that have no medical concepts before the target diagnosis
                        else:    
                            exclude.append(0)
                            te = len(feed_events)
                            
                            patient_seq = 0
                            patient_seq_all_disease.append(patient_seq)
                            newfeed = []
                            newfeed1 = []
                            newfeed2 = []
                            patient_seq = []
                            for n in range(len(feed_events)):
                                newfeed.append(feed_events[n].replace("_",r""))
                            for y in range(len(newfeed)):
                                newfeed1.append(newfeed[y].replace(".",r""))
                            for z in range(len(newfeed1)):
                                newfeed2.append(newfeed1[z].replace("/",r"q"))
                            feed_events = newfeed2

                            feed = ' '.join(feed_events)
                            patient_seq = bc.encode([feed]) #Generate BERT sentence embedding for all events in history
                            patient_seq_all_disease_1.append(patient_seq)

                            
                            

                    else: 
                        exclude.append(0)
                        te = len(feed_events)
                        
                        patient_seq = 0
                        patient_seq_all_disease.append(patient_seq)
                        newfeed = []
                        newfeed1 = []
                        newfeed2 = []
                        patient_seq = []
                        for n in range(len(feed_events)):
                            newfeed.append(feed_events[n].replace("_",r""))
                        for y in range(len(newfeed)):
                            newfeed1.append(newfeed[y].replace(".",r""))
                        for z in range(len(newfeed1)):
                            newfeed2.append(newfeed1[z].replace("/",r"q"))
                        feed_events = newfeed2

                        feed = ' '.join(feed_events)
                        patient_seq = bc.encode([feed]) #Generate BERT sentence embedding for all events in history
                        patient_seq_all_disease_1.append(patient_seq)

        patient_seq_all_disease = []
        
    
        flat_embeddings = list(itertools.chain(*patient_seq_all_disease_1))

        ## Create a dataframe for all the patient vectors
        data_disease_1=pd.DataFrame(flat_embeddings)

        ## Create the train labels, make sure to get rid of labels where we exclude patients
        disease_final = []
        for i in range(0,len(exclude)):
            if exclude[i] == 0:
                disease_final.append(disease_final_ori[i])


        data_disease_1[test[q]] = disease_final


        X_disease_1=data_disease_1.iloc[:,0:100]
        Y_disease = data_disease_1.iloc[:,100]

        return X_disease_1, Y_disease, data_disease_1

    X_train_disease_1, Y_train_disease, data_disease_1 = generate_patient_disease_vectors(train_files, events, bert_dict)
    X_disease_test_1, Y_disease_test, data_disease_test_1 = generate_patient_disease_vectors(valid_files, events, bert_dict)
    



/opt/anaconda3/envs/[py3.6]/lib/python3.6/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


In [8]:
## Save dataframe in xlsx file

import openpyxl
import pandas as pd
data_disease_1.to_excel('/users/emily/Documents/sentence/ckd_train.xlsx')
data_disease_test_1.to_excel('/users/emily/Documents/sentence/ckd_test.xlsx')